In [13]:
client.models.list().page 

[Model(name='models/embedding-gecko-001', display_name='Embedding Gecko', description='Obtain a distributed representation of a text.', version='001', endpoints=None, labels=None, tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None), input_token_limit=1024, output_token_limit=1, supported_actions=['embedText', 'countTextTokens'], default_checkpoint_id=None, checkpoints=None),
 Model(name='models/gemini-1.0-pro-vision-latest', display_name='Gemini 1.0 Pro Vision', description='The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.', version='001', endpoints=None, labels=None, tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None), input_token_limit=12288, output_token_limit=4096, supported_actions=['generateContent', 'countTokens'], default_checkpoint_id=None, checkpoints=None),
 Model(name='models/gem

In [26]:
import re
import ast
from typing import List, Any

def extract_and_parse_multi_class_prompt_list(code: str) -> List[str]:
    """
    From a string of Python code, finds the first occurrence of
        = [ ... ]
    and parses that bracketed literal into a List[str].

    The string in the list contains a placeholder <>,

    Raises:
        ValueError if no list literal is found or it’s malformed,
                   or if the parsed list does not contain only strings.
    """
    # 1) Grab everything from the first '=' up to the matching ']'
    # This regex is made more robust to handle various whitespace and content within the list.
    m = re.search(r'=\s*(\[[^\]]*\])', code, re.DOTALL)
    if not m:
        raise ValueError("No list literal found after an '=' in the code")
    list_str = m.group(1)

    # 2) Safely evaluate it (only literals)
    try:
        data: Any = ast.literal_eval(list_str)
    except (SyntaxError, ValueError) as e:
        raise ValueError(f"Malformed list literal: {e}")

    # 3) Validate shape: ensure it's a list of strings
    if not isinstance(data, list) or not all(isinstance(item, str) for item in data):
        raise ValueError(
            "Parsed object is not a list of strings, or contains non-string elements."
        )

    # 4) Return the list of strings
    return data

code = """prompts: list[str] = [
    "An histopathological image of a <> region.",
    "The central area shows features consistent with <> tissue.",
    "This lymph node patch displays characteristics of a <> infiltrate.",
    "Observe the cellular morphology within this <> area.",
    "The predominant feature of this patch is <> architecture.",
    "The nuclear pleomorphism is indicative of a <> process.",
    "The staining intensity suggests a <> composition.",
    "The density of cells in this region is typical of <> tissue.",
    "The presence of mitotic figures suggests a <> state.",
    "The stromal reaction is characteristic of <> involvement.",
    "The overall pattern is consistent with a <> response.",
    "This patch exhibits features of <> lymphocytes.",
    "The arrangement of cells suggests a <> origin.",
    "The size and shape of nuclei are consistent with <> cells.",
    "The cytoplasm staining is indicative of a <> phenotype.",
    "The presence of necrosis suggests a <> condition.",
    "The inflammatory response appears to be <>.",
    "The vascularity within this region is typical of <> tissue.",
    "The fibrosis observed is consistent with <> changes.",
    "The cellularity is markedly increased in this <> area.",
    "The nuclei are hyperchromatic in this <> region.",
    "The architectural pattern is disrupted in the <> tissue.",
    "The stromal component is prominent in this <> patch.",
    "The overall impression is that of a <> process.",
    "The cells exhibit atypical features consistent with <> cells.",
    "The presence of giant cells suggests a <> reaction.",
    "The distribution of cells is uniform in this <> area.",
    "The nuclei are round and regular in this <> tissue.",
    "The cytoplasm is abundant in these <> cells.",
    "The staining is pale in this <> region.",
    "The cellular borders are distinct in this <> area.",
    "The nuclei are vesicular in this <> tissue.",
    "The architectural pattern is well-preserved in the <> tissue.",
    "The stromal component is sparse in this <> patch.",
    "The overall impression is that of a <> response.",
    "The cells exhibit features of <> differentiation.",
    "The presence of lymphocytes suggests a <> response.",
    "The distribution of cells is irregular in this <> area.",
    "The nuclei are irregular in shape in this <> tissue.",
    "The cytoplasm is scant in these <> cells.",
    "The staining is intense in this <> region.",
    "The cellular borders are indistinct in this <> area.",
    "The nuclei are densely packed in this <> tissue.",
    "The architectural pattern is disorganized in the <> tissue.",
    "The stromal component is dense in this <> patch.",
    "The overall impression is that of a <> process.",
    "The cells show evidence of <> activity.",
    "The presence of plasma cells suggests a <> response.",
    "The arrangement of cells is follicular in this <> area.",
    "The nuclei are enlarged in this <> tissue.",
    "The cytoplasm is eosinophilic in these <> cells.",
    "The staining is granular in this <> region.",
    "The cellular morphology is consistent with <> cells.",
    "The nuclei are binucleated in this <> tissue.",
    "The architectural pattern is nodular in the <> tissue."
]"""
extract_and_parse_multi_class_prompt_list(code=code)

['An histopathological image of a <> region.',
 'The central area shows features consistent with <> tissue.',
 'This lymph node patch displays characteristics of a <> infiltrate.',
 'Observe the cellular morphology within this <> area.',
 'The predominant feature of this patch is <> architecture.',
 'The nuclear pleomorphism is indicative of a <> process.',
 'The staining intensity suggests a <> composition.',
 'The density of cells in this region is typical of <> tissue.',
 'The presence of mitotic figures suggests a <> state.',
 'The stromal reaction is characteristic of <> involvement.',
 'The overall pattern is consistent with a <> response.',
 'This patch exhibits features of <> lymphocytes.',
 'The arrangement of cells suggests a <> origin.',
 'The size and shape of nuclei are consistent with <> cells.',
 'The cytoplasm staining is indicative of a <> phenotype.',
 'The presence of necrosis suggests a <> condition.',
 'The inflammatory response appears to be <>.',
 'The vascularit

In [24]:
import util

from API_KEY import GEMINI_API_KEY
import re
import ast
from typing import List, Any

from google import genai

client = genai.Client(api_key=GEMINI_API_KEY)

for i in range(1):
    response = client.models.generate_content(
        model="gemma-3-27b-it", contents="""The task is to generate distinct textual description prompts of visual discriminative features to describe the central region of an histopathological image patch. The patch is extracted from an H&E‑stained whole‑slide image of a lymph node section. The patch can be either tumor or normal tissue.
Each prompt should have a placeholder <> that can be replaced with either "tumor" or "normal". I will fill in the placeholder later.
Here are the best performing pairs in ascending order. High scores indicate higher quality visual discriminative features.
Current Top 10 Prompts:
1. An histopathological image of a <> lymph node section. Score: 80
2. A high-magnification view of <> tissue from a lymph node biopsy. Score: 75
3. Characterize the stromal components and their relationship to the <> Score: 74

Write 10 new prompts that are different to from the old ones and has a score as high as possible. Formulate a strategy
Output as python code in the format - prompts: list[prompt:str]. Let's think step-by-step
""")
    print(response.text)

Okay, let's formulate a strategy to generate high-scoring prompts, and then implement it in Python.

**Strategy:**

The current top prompts focus on broad descriptions of the tissue and its context (lymph node, biopsy).  To improve, we need to:

1. **Specificity:**  Move beyond just "tissue" and mention cellular characteristics. Histopathology is about *cells*.
2. **Discriminative Features:** Focus on features that *differentiate* tumor from normal.  This means things like cellularity, nuclear features (size, shape, staining), mitotic activity, and architectural patterns.
3. **Actionable Verbs:** Use verbs that encourage detailed description ("assess", "evaluate", "describe").  "Characterize" is good, but we can do better.
4. **Avoid Redundancy:**  Ensure the new prompts are distinct from the existing ones.
5. **Contextualize:** Keep the lymph node context, as that's important.

Here are some ideas for features to include:

*   **Cellularity:**  Dense vs. sparse cell populations.
*   *

In [ ]:
import util

from API_KEY import GEMINI_API_KEY
import re
import ast
from typing import List, Any

from google import genai

client = genai.Client(api_key=GEMINI_API_KEY)

for i in range(1):
    response = client.models.generate_content(
        model="gemma-3-27b-it", contents="""The task is to generate 20 distinct textual description prompts of visual discriminative features to describe the central region of an histopathological image patch. The patch is extracted from an H&E‑stained whole‑slide image of a lymph node section. The patch can be either tumor or normal tissue.
Each prompt should have the multiple options for each class 'normal' and 'tumor'. In each prompt, along with the medical concept add an selector in the format: '[normal observation, tumor observation]'. The text should be meaningful after when one of the options is filled.

Eg: The stromal component is ['fibrotic', 'edematous']
Output as python code in the format - prompts: list[prompt:str]. Let's think step-by-step
""")
    print(response.text)

Okay, let's break this down. We need to generate 20 prompts, each designed to elicit a description of a histopathological image patch, focusing on the central region. Each prompt will contain a medical concept and a selector with two options – one typical of normal lymph node tissue and one typical of tumor involvement.  The prompts should be phrased so that the resulting sentence makes sense regardless of which option is chosen.  We'll use Python to store these as a list of strings.

Here's a step-by-step thought process and then the Python code:

1. **Identify Key Histopathological Features:**  We need to think about what features pathologists look at in lymph node H&E sections. These include:
    *   Cellularity (how many cells are present)
    *   Lymphocyte morphology (size, shape, nucleus, cytoplasm)
    *   Presence of mitoses (cell division)
    *   Presence of necrosis (cell death)
    *   Capsular integrity
    *   Follicular structure (if present)
    *   Stromal characteris

ChatGPT

In [8]:
from openai import AzureOpenAI
from API_KEY import CHATGPT_API_KEY, CHATGPT_ENDPOINT
import json

# Initialize Azure OpenAI client with key-based authentication
client = AzureOpenAI(
    azure_endpoint=CHATGPT_ENDPOINT,
    api_key=CHATGPT_API_KEY,
    api_version="2025-01-01-preview",
)

# Prepare the chat prompt
prompt = """Give 50 distinct textual descriptions of pairs of visual discriminative features to identify whether the central region of a histopathological image patch contains tumor tissue or not. The patch is extracted from an H&E‑stained whole‑slide image of a lymph node section. Only provide the output as Python code in the following format: prompts = list[tuple[negative: str, positive: str]]. Let's think step-by-step"""
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": prompt
            }
        ]
    }
]

# Generate the completion
completion = client.chat.completions.create(
    model='gpt-4.1',
    messages=messages,
    max_tokens=5000,
    temperature=1.0,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False
)

# parse the response to JSON, then convert to dictionary object
response = json.loads(completion.to_json())['choices'][0]['message']['content']

print(response)


```python
prompts = [
    ("Central region shows uniform, small, round nuclei with inconspicuous nucleoli", "Central region displays enlarged, pleomorphic nuclei with prominent nucleoli"),
    ("No evident mitotic figures are present in the center", "Numerous abnormal mitotic figures are visible in the center"),
    ("Cytoplasm is evenly stained and not abundant", "Cells show abundant, pale or eosinophilic cytoplasm"),
    ("Normal lymphoid architecture with follicular patterns is preserved", "Architectural distortion with loss of normal follicular pattern"),
    ("Central cells have high nuclear-to-cytoplasmic ratio", "Central cells exhibit variable nuclear-to-cytoplasmic ratios"),
    ("Few or no apoptotic bodies are seen", "Frequent apoptotic bodies are scattered in the region"),
    ("Lymphocytes in center are small and regular", "Atypical, irregularly shaped lymphocytes predominate in center"),
    ("Central tissue appears non-necrotic", "Central tissue contains large areas of nec